In [1]:
import os
import pandas as pd
import numpy as np
import os
import glob
import glob2
import pathlib
import re
import pyperclip
import email
from datetime import date, timedelta

### Funcitions for sliding time window

In [2]:
def move_two_weeks(start_week, next_two_week):
    start_week = next_two_week
    next_two_week = next_two_week + timedelta(weeks=2)
    return start_week, next_two_week


def reset_weeks(start_week, next_two_week):
    start_week = min_date.date()
    next_two_week = start_week + timedelta(weeks=2)
    return start_week, next_two_week

### Loading dataset, creating a list of employee emails for future reference

In [3]:
employee_names = [
    "Phillip Allen", "John Arnold", "harry.arora", "Robert Badeer",
    "Susan Bailey", "Eric Bass", "Don Baughman", "Sally Beck", "Robert Benson",
    "Lynn Blair", "Sandra Brawner", "Rick Buy", "Larry Campbell",
    "Mike Carson", "Michelle Cash", "Monika Causholli", "Shelley Corman",
    "Sean Crandall", "Martin Cuilla", "jeff_dasovich", "Dana Davis",
    "Craig Dean", "David Delainey", "James Derrick", "Stacy Dickson",
    "Lindy Donoho", "Tom Donohoe", "Chris Dorland", "Frank Ermis",
    "Daren Farmer", "Mary Fischer", "John Forney", "Drew Fossum", "Lisa Gang",
    "Randall Gay", "Tracy Geaccone", "Chris Germany", "doug.gilbert-smith",
    "Darron Giron", "john.griffith", "mike.grigsby", "Mark Guzman",
    "Mark Haedicke", "Mary Hain", "Steven Harris", "Rod Hayslett",
    "Marie Heard", "Scott Hendrickson", "Juan Hernandez", "John Hodge",
    "Keith Holst", "Stanley Horton", "Kevin Hyatt", "Dan Hyvl", "Tana Jones",
    "Vince Kaminski", "Steven Kean", "Peter Keavey", "Kam Keiser", "Jeff King",
    "Louise Kitchen", "Tori Kuykendall", "John Lavorato", "Kenneth Lay",
    "Matthew Lenhart", "Andrew Lewis", "Eric Linder", "Michelle Lokay",
    "Teb Lokey", "Phillip Love", "Paul Lucci", "mike.maggi", "Kay Mann",
    "Thomas Martin", "larry.may", "Danny McCarty", "Mike McConnell",
    "brad.mckay", "Jonathan Mckay", "Errol McLaughlin", "steven.merris",
    "Albert Meyers", "l..mims", "matt.motley", "Scott Neal", "Gerald Nemec",
    "Stephanie Panus", "Joe Parks", "Susan Pereira", "Debra Perlingiere",
    "Vladi Pimenov", "Phillip Platter", "Kevin Presto", "Joe Quenet",
    "Dutch Quigley", "Bill Rapp", "Jay Reitmeyer", "Cooper Richey",
    "Andrea Ring", "Richard Ring", "Robin Rodrigue", "Benjamin Rogers",
    "Kevin Ruscitti", "Elizabeth Sager", "Eric Saibi", "Holden Salisbury",
    "Monique Sanchez", "Richard Sanders", "Diana Scholtes",
    "Darrell Schoolcraft", "Jim Schwieger", "Susan Scott", "Cara Semperger",
    "Sara Shackleton", "Jeffrey Shankman", "Richard Shapiro", "Hunter Shively",
    "jeff.skilling", "Ryan Slinger", "Matt Smith", "Geir Solberg",
    "steven.south", "Theresa Staab", "Carol Clair", "James Steffes",
    "Joe Stepenovitch", "Chris Stokley", "geoff.storey", "Fletcher Sturm",
    "Mike Swerzbin", "Kate Symes", "Mark Taylor", "Jane Tholt", "Paul Thomas",
    "Judy Townsend", "Barry Tycholiz", "Kim Ward", "Kimberly Watson",
    "Charles Weldon", "Greg.Whalley", "Stacey White", "Mark Whitt",
    "Jason Williams", "bill.williams", "Jason Wolfe", "Paul Y'Barbo",
    "Andy Zipper", "John Zufferli"
]

employee_emails = [
    x.lower().replace(" ", ".") + "@enron.com" for x in employee_names
]
ds = pd.read_csv("./dataset.csv")
pattern = "|".join(employee_emails)

# Filtering dataframe to only contain
# emails that are sent by any of the employees
ds = ds[ds["from"].str.contains(pattern)]
ds["date"] = pd.to_datetime(ds["date"])
min_date = ds["date"].min()
max_date = ds["date"].max()
start_week = min_date.date()
next_two_week = start_week + timedelta(weeks=2)

### Chunk dataset into periods of two weeks and save them as `.csv` locally

In [4]:
start_week, next_two_week = reset_weeks(start_week, next_two_week)
count = 0
while start_week < max_date:
    # print(count)
    # print(start_week, next_two_week)
    ss = ds[(ds["date"] >= pd.to_datetime(start_week))
            & (ds["date"] < pd.to_datetime(next_two_week))]
    start_week, next_two_week = move_two_weeks(start_week, next_two_week)
    if ss.count()["id"] > 0:
        ss.to_csv(f"./out/period-0000{count}.csv")
    count += 1

C:\Users\Alireza Vaezi\AppData\Local\Temp\ipykernel_21796\1975823543.py:3: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  while start_week < max_date:


### Loading each period and making adj-matrix for each and saving locally as numpy arrays

In [10]:
chunk_directory = "./out/*.csv"

all_paths = glob.glob(chunk_directory)
period = pd.DataFrame(columns=['id', 'date', 'from', 'to', 'cc', 'bcc'])
adj_matrix = np.zeros((148, 148), dtype=int)



In [12]:
for i, k in enumerate(all_paths):
    df = pd.read_csv(k)
    # print(df["from"].unique())
    for _, row in df.iterrows():

        from_index = employee_emails.index(row['from'])
        # print(from_index)

        to_list = row['to'].split(',') if pd.notna(row['to']) else []
        # print(to_list)
        for to in to_list:
            if to in employee_emails:
                to_index = employee_emails.index(to)
                adj_matrix[from_index, to_index] += 1
                adj_matrix[to_index, from_index] += 1

        adj_matrix_df = pd.DataFrame(adj_matrix,
                                     columns=employee_emails,
                                     index=employee_emails)
        adj_matrix_df.to_csv(k.replace(".csv", "-adj_matrix.csv"))

        t = adj_matrix_df.to_numpy()
        with open(k.replace(".csv", "-adj_matrix.npy"), 'wb') as f:
            np.save(f, t)


KeyError: 'from'